In [ ]:
!pip install pandas
!pip install requests
!pip install sqlalchemy
!pip install flask


In [ ]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy as sql
from sqlalchemy import inspect
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker, Session, relationship

#### create database

In [ ]:
engine = sql.create_engine("sqlite:///sports_analysis.sqlite")
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

In [ ]:
class Team(Base):
    __tablename__ = 'teams'
    id = Column(String, primary_key=True)
    name = Column(String)
    abbreviation = Column(String)
    market = Column(String)

    players = relationship("Player", back_populates="team")

class Player(Base):
    __tablename__ = 'players'
    id = Column(String, primary_key=True)
    full_name = Column(String)
    position = Column(String)
    jersey_number = Column(String)
    team_id = Column(String, ForeignKey('teams.id'))

    team = relationship("Team", back_populates="players")
    stats = relationship("PlayerStat", back_populates="player")

class PlayerStat(Base):
    __tablename__ = 'player_stats'
    id = Column(Integer, primary_key=True)
    player_id = Column(String, ForeignKey('players.id'))
    season_id = Column(String)
    games_played = Column(Integer)
    hits = Column(Integer)
    home_runs = Column(Integer)
    rbi = Column(Integer)

    player = relationship("Player", back_populates="stats")
Base.metadata.create_all(engine)
    

In [ ]:
def fetch_players(api_key, season_id):
    url = f"https://api.sportradar.com/mlb/trial/v8/en/seasons/{season_id}/statistics.json?api_key={api_key}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()
    
def fetch_season_stats(api_key, season_id):
    url = f"https://api.sportradar.com/mlb/trial/v8/en/seasons/{season_id}/statistics.json?api_key={api_key}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

In [ ]:
def insert_players(data):
    teams = data.get('teams', [])
    
    for team in teams:
        team_name = team.get('name', '')
        players = team.get('players', [])
        
        for player in players:
            player_id = player.get('id')
            name = player.get('full_name')
            position = player.get('primary_position', {}).get('abbreviation', '')
            number = player.get('jersey_number', '')

            new_player = Player(
                id=player_id,
                full_name=name,
                team=team_name,
                position=position,
                jersey_number=number
            )
            session.merge(new_player)  # merge = insert or update
    session.commit()


In [ ]:
if __name__ == "__main__":
    API_KEY = 'NYii5TfXY5KzhPP7RPf2Wj5eMOxiZPqFI1jDuKMB'
    SEASON_ID = '2025/REG'  # Example season

    data = fetch_players(API_KEY, SEASON_ID)
    insert_players(data)
    print("Players added to database.")


In [ ]:
api_key: "NYii5TfXY5KzhPP7RPf2Wj5eMOxiZPqFI1jDuKMB"


url = "https://api.sportradar.com/mlb/trial/v8/en/players/80de60c9-74e3-4a50-b128-b3dc7456a254/profile.json"

headers = {
    "accept": "application/json",
    "x-api-key": "NYii5TfXY5KzhPP7RPf2Wj5eMOxiZPqFI1jDuKMB"
}

response = requests.get(url, headers=headers).json()

print(response)

In [ ]:
# GET https://api.sportradar.com/mlb/trial/v8/en/seasons/2025REG/statistics.json?api_key=YOUR_API_KEY

In [ ]:
response

In [ ]:
#convert to json, call keys, dataframe and vis

In [ ]:
response.keys()

In [ ]:
response['player'].keys()

In [ ]:
response["player"][2]

In [ ]:
#example if you have multiplayers
rows = []
players_list = response["player"]

for players in players_list:
    Player_id = players['id'],
    status = players['status']
    primary_position = players['primary_position']
    ...

    rows.append({'id': Player_id,
                 'status': status,
                 'primary_position': primary_position
                 ....
                
                })
rows_to_df = pd.DataFrame(rows)

In [ ]:
rows = []
rows.append({'id': response["player"]['id'],
                 'status': response["player"]['status'],
                 'primary_position': response["player"]['primary_position'] })
rows_to_df = pd.DataFrame(rows)

In [ ]:
rows_to_df

In [ ]:
dic1

In [ ]:
dic1.keys()

In [ ]:
items_list = list(dic1.items())
print(items_list)

In [ ]:
for item in items_list:
    print(type(item),item)

In [ ]:
my_dict= dict(items_list)

In [ ]:
print(my_dict)

In [ ]:
df = pd.DataFrame(my_dict)

In [ ]:
"""
url = "https://api.sportradar.com/mlb/trial/v8/en/league/teams.json"

headers = {
    "accept": "application/json",
    "x-api-key": "NYii5TfXY5KzhPP7RPf2Wj5eMOxiZPqFI1jDuKMB"
}

response2 = requests.get(url, headers=headers).json()

print(response2)
"""

In [ ]:
#def player_search(): ; can choose via first_name or last_name
# player input('Search name')